*Based on [Enhancing RAG-based application accuracy by constructing and leveraging knowledge graphs](https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/)*

# Libraries

In [1]:
import os
from typing import Any, List

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.chains import (create_history_aware_retriever,
                              create_retrieval_chain)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage
from langchain.text_splitter import TokenTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.graphs import Neo4jGraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Neo4jVector
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.retrievers import BaseRetriever, Document
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from src.async_llm_transformer import AsyncLLMGraphTransformer

In [2]:
# Load environment variables
load_dotenv()

True

# Data

### Load and split data

For this notebook we will be using the [AWS Generative AI Competency checklist](https://apn-checklists.s3.amazonaws.com/competency/generative-ai/consulting/Cks9Pu3tv.html) as the data for the RAG.

In [3]:
# Using bs4 to parse the AWS Generative AI Competency webpage
loader = WebBaseLoader("https://apn-checklists.s3.amazonaws.com/competency/generative-ai/consulting/Cks9Pu3tv.html")
docs = loader.load()

In [4]:
# Splitting the documents into chunks
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
len(documents)

17

### Create Graph Database (Neo4jGraph) connection

In [6]:
# Initialize the connection with Neo4j graph database
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
)

## Constructing the graph

#### Converting to graph documents

We will use `LLMGraphTransformer` class in order to convert documents to graph documents. However, the original `LLMGraphTransformer` takes quite a while because of its synchronous logic. We can adapt it to execute it asynchronous with small changes:
```
class AsyncLLMGraphTransformer(LLMGraphTransformer):
    async def aprocess_response(self, document: Document) -> GraphDocument:
        """
        Asynchronously processes a single document, transforming it into a graph document.
        """
        text = document.page_content
        raw_schema = await self.chain.ainvoke({"input": text})
        nodes = [map_to_base_node(node) for node in raw_schema.nodes] if raw_schema.nodes else []
        relationships = (
            [map_to_base_relationship(rel) for rel in raw_schema.relationships] if raw_schema.relationships else []
        )

        if self.strict_mode and (self.allowed_nodes or self.allowed_relationships):
            if self.allowed_relationships and self.allowed_nodes:
                nodes = [node for node in nodes if node.type in self.allowed_nodes]
                relationships = [
                    rel
                    for rel in relationships
                    if rel.type in self.allowed_relationships
                    and rel.source.type in self.allowed_nodes
                    and rel.target.type in self.allowed_nodes
                ]
            elif self.allowed_nodes and not self.allowed_relationships:
                nodes = [node for node in nodes if node.type in self.allowed_nodes]
                relationships = [
                    rel
                    for rel in relationships
                    if rel.source.type in self.allowed_nodes and rel.target.type in self.allowed_nodes
                ]
            if self.allowed_relationships and not self.allowed_nodes:
                relationships = [rel for rel in relationships if rel.type in self.allowed_relationships]

        return GraphDocument(nodes=nodes, relationships=relationships, source=document)

    async def aconvert_to_graph_documents(self, documents: Sequence[Document]) -> List[GraphDocument]:
        """
        Asynchronously convert a sequence of documents into graph documents.
        """
        tasks = []
        for document in documents:
            task = asyncio.create_task(self.aprocess_response(document))
            tasks.append(task)

        results = await asyncio.gather(*tasks)
        return results
```

In [7]:
# Load OpenAIs GPT-4 and LLMGraphTransformer class
llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
llm_transformer = AsyncLLMGraphTransformer(llm=llm)

In [10]:
# Convert the documents into graph documents
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents=documents)
graph_documents

[GraphDocument(nodes=[Node(id='Aws Partner Network', type='Organization'), Node(id='Aws Generative Ai Competency', type='Program'), Node(id='Service Offering Validation Checklist', type='Document'), Node(id='Aws Specialization Programs', type='Program'), Node(id='Aws Competency Partner Validation Checklist', type='Document'), Node(id='Aws Partners', type='Organization'), Node(id='Aws Partner Development Representative', type='Role'), Node(id='Aws Partner Development Manager', type='Role'), Node(id='March 2024', type='Date')], relationships=[Relationship(source=Node(id='Aws Generative Ai Competency', type='Program'), target=Node(id='Service Offering Validation Checklist', type='Document'), type='HAS_DOCUMENT'), Relationship(source=Node(id='Aws Partner Network', type='Organization'), target=Node(id='Aws Generative Ai Competency', type='Program'), type='OFFERS'), Relationship(source=Node(id='Aws Specialization Programs', type='Program'), target=Node(id='Aws Competency Partner Validation C

#### What is happening?
You can take a look at an example in [LangSmith](https://smith.langchain.com/public/e6434997-e31d-4bdf-8ec5-367b76208252/r). I also transcribe the exact prompts in the following cell.

In [11]:
system = "# Knowledge Graph Instructions for GPT-4\n## 1. Overview\nYou are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.\nTry to capture as much information from the text as possible without sacrifing accuracy. Do not add any information that is not explicitly mentioned in the text\n- **Nodes** represent entities and concepts.\n- The aim is to achieve simplicity and clarity in the knowledge graph, making it\naccessible for a vast audience.\n## 2. Labeling Nodes\n- **Consistency**: Ensure you use available types for node labels.\nEnsure you use basic or elementary types for node labels.\n- For example, when you identify an entity representing a person, always label it as **'person'**. Avoid using more specific terms like 'mathematician' or 'scientist'  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.\n- **Relationships** represent connections between entities or concepts.\nEnsure consistency and generality in relationship types when constructing knowledge graphs. Instead of using specific and momentary types such as 'BECAME_PROFESSOR', use more general and timeless relationship types like 'PROFESSOR'. Make sure to use general and timeless relationship types!\n## 3. Coreference Resolution\n- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.\nIf an entity, such as \"John Doe\", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., \"Joe\", \"he\"),always use the most complete identifier for that entity throughout the knowledge graph. In this example, use \"John Doe\" as the entity ID.\nRemember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.\n## 4. Strict Compliance\nAdhere to the rules strictly. Non-compliance will result in termination."
user = "Tip: Make sure to answer in the correct format and do not include any explanations. Use the given format to extract information from the following input: <Chunk of text>"
structured_output_schema = '''def create_simple_model(
    node_labels: Optional[List[str]] = None, rel_types: Optional[List[str]] = None
) -> Any:
    """
    Simple model allows to limit node and/or relationship types.
    Doesn't have any node or relationship properties.
    """

    class SimpleNode(BaseModel):
        """Represents a node in a graph with associated properties."""

        id: str = Field(description="Name or human-readable unique identifier.")
        type: str = optional_enum_field(
            node_labels, description="The type or label of the node."
        )

    class SimpleRelationship(BaseModel):
        """Represents a directed relationship between two nodes in a graph."""

        source: SimpleNode = Field(description="The source node of the relationship.")
        target: SimpleNode = Field(description="The target node of the relationship.")
        type: str = optional_enum_field(
            rel_types, description="The type of the relationship.", is_rel=True
        )

    class DynamicGraph(BaseModel):
        """Represents a graph document consisting of nodes and relationships."""

        nodes: Optional[List[SimpleNode]] = Field(description="List of nodes")
        relationships: Optional[List[SimpleRelationship]] = Field(
            description="List of relationships"
        )

    return DynamicGraph
'''
example = "----------------------------- SYSTEM MESSAGE -----------------------------\n\n" \
f"{system}\n\n" \
"----------------------------- USER MESSAGE -----------------------------\n\n" \
f"{user}\n\n" \
"----------------------------- STRUCTURED OUTPUT SCHEMA  -----------------------------\n\n" \
f"{structured_output_schema}\n\n"
print(example)

----------------------------- SYSTEM MESSAGE -----------------------------

# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Try to capture as much information from the text as possible without sacrifing accuracy. Do not add any information that is not explicitly mentioned in the text
- **Nodes** represent entities and concepts.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it
accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use available types for node labels.
Ensure you use basic or elementary types for node labels.
- For example, when you identify an entity representing a person, always label it as **'person'**. Avoid using more specific terms like 'mathematician' or 'scientist'  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the 

#### Add documents to the database

In [12]:
# Add graph documents to the database
graph.add_graph_documents(
    graph_documents=graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [13]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = "id"
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

### Retrieval

As defined in https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/ we will be building a retrieval approach that combines vector and keyword indexes with graph retrieval for RAG.

#### 1. Vector + Keyword Index

In [14]:
vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [15]:
vector_index.similarity_search("What are the Generative AI Practice Requirements?")

[Document(page_content="\ntext:  format. Links to the appropriate Self-Assessment Spreadsheet can be found at the top of this page.\n\nAWS Partner must complete all sections of the Self-Assessment Spreadsheet. For competency with multiple categories, AWS Partners will fill in details for the chosen application Category and mark other Categories as N/A.\nCompleted Self-Assessment Spreadsheet must be uploaded at the time of submitting an application in APN Partner Central.\nIt is recommended that AWS Partner have their AWS Partner Solution Architect, Partner Development Representative (PDR), or Partner Development Manager (PDM) review the completed Self-Assessment Spreadsheet before submitting to AWS. The purpose of this is to ensure the AWS Partner’s AWS team is engaged and working to provide recommendations prior to the validation and to help ensure a positive validation experience.\n\n\n\n\n\n\n\n\n\nCommon AWS Partner Practice Requirements\nThe following requirements validate the mec

#### 2. Graph Retrieval

We create a full-text index named `entity` in the Neo4j graph database in order to do efficient searching and querying of text data in the database.

In [16]:
# Create a full-text index named "entity" on the nodes labeled "__Entity__"
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

We will use structured ouput again to extract the entities in the user query:

In [17]:
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(..., description="All the person, organization, or business entities that appear in the text")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are extracting organization and person entities from the text."),
    ("human", "Use the given format to extract information from the following input: {question}")
])

entity_chain = prompt | llm.with_structured_output(Entities)

/home/jjmov99/agentic-knowledge-graph-rag/.venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Define auxiliar functions for the graph retriever. 

In [18]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [19]:
print(structured_retriever("What are the Generative AI Practice Requirements?"))

Aws Partner Network - OFFERS -> Aws Generative Ai Competency
Aws Generative Ai Competency - HAS_DOCUMENT -> Service Offering Validation Checklist
Service Offering Validation Checklist - PUBLISHED_ON -> March 2024
Aws Specialization Programs - HAS_DOCUMENT -> Aws Competency Partner Validation Checklist
Aws Partners - APPLY_FOR -> Aws Generative Ai Competency
Aws Partners - CONTACT -> Pdr
Aws Partners - CONTACT -> Pdm
Aws Partners - COMPLETE -> Self-Assessment Spreadsheet
Aws Partners - SUBMIT_APPLICATION -> Apn Partner Central
Aws Partners - PREPARE -> Technical Validation
Aws Partners - USE -> Checklist
Aws Partners - HAVE -> Aws Certified Engineers/Architects
Aws Partners - HAVE -> Operations Manager
Aws Partners - HAVE -> Business Development Executive
Self-Assessment Spreadsheet - RELATED_TO -> Aws Specialization Designations
Self-Assessment Spreadsheet - MUST_BE_UPLOADED_TO -> Apn Partner Central
Aws Generative Ai Services Partners - DELIVER -> Aws Generative Ai Competency Definiti

#### 3. Final Retriever


In [20]:
class FinalRetriever(BaseRetriever):
    structured_retriever: Any
    vector_index: Any

    def get_relevant_documents(self, query: str) -> str:
        # Graph retrieval
        structured_results = [Document(page_content=self.structured_retriever(query))]
        # Vector + Keyword retrieval
        unstructured_results = self.vector_index.similarity_search(query)
        return structured_results + unstructured_results

In [21]:
final_retriever = FinalRetriever(
    structured_retriever=structured_retriever,
    vector_index=vector_index
)
final_retriever.get_relevant_documents("What are the Generative AI Practice Requirements?")

[Document(page_content='Aws Partner Network - OFFERS -> Aws Generative Ai Competency\nAws Generative Ai Competency - HAS_DOCUMENT -> Service Offering Validation Checklist\nService Offering Validation Checklist - PUBLISHED_ON -> March 2024\nAws Specialization Programs - HAS_DOCUMENT -> Aws Competency Partner Validation Checklist\nAws Partners - APPLY_FOR -> Aws Generative Ai Competency\nAws Partners - CONTACT -> Pdr\nAws Partners - CONTACT -> Pdm\nAws Partners - COMPLETE -> Self-Assessment Spreadsheet\nAws Partners - SUBMIT_APPLICATION -> Apn Partner Central\nAws Partners - PREPARE -> Technical Validation\nAws Partners - USE -> Checklist\nAws Partners - HAVE -> Aws Certified Engineers/Architects\nAws Partners - HAVE -> Operations Manager\nAws Partners - HAVE -> Business Development Executive\nSelf-Assessment Spreadsheet - RELATED_TO -> Aws Specialization Designations\nSelf-Assessment Spreadsheet - MUST_BE_UPLOADED_TO -> Apn Partner Central\nAws Generative Ai Services Partners - DELIVER 

### Testing the retriever

#### 1. Retrieval Chain

In [22]:
template = """Answer the following question based only on the provided context. If the context is not enough to answer the question, return "I don't know".

<context>
{context}
</context>

Question: {input}"""
prompt = ChatPromptTemplate.from_template(template=template)
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(retriever=final_retriever, combine_docs_chain=document_chain)

In [23]:
response = retrieval_chain.invoke({"input": "What are the Generative AI Practice Requirements?"})["answer"]
print(response)

The Generative AI Practice Requirements for AWS Partners include:

1. **Customer Onboarding, Adoption Strategy, and Implementation Plan of Generative AI**: The partner evaluates the customer's maturity for generative AI to craft a strategy that aligns with the customer's processes, skills, organizational structure, data available, industry, use cases, budget, and tolerance to risk. This includes use case detection, data sources, data quality validation, rapid experimentation, organizational structure updates, generative AI projects roadmap definition, features backlog, enablement, iterative processes, and bottom-up innovation. Evidence expected includes a written description of the generative AI methodology developed by the partner, highlighting how they evaluate clients' readiness for generative AI, the process for evaluating the outcomes of existing generative AI projects, and how the partner identifies potential use cases for generative AI within a client's business operations.

2. 

#### 2. Conversation Retrieval Chain


In [24]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm=llm, retriever=final_retriever, prompt=prompt)
retrieval_chain = create_retrieval_chain(retriever=retriever_chain, combine_docs_chain=document_chain)

In [26]:
chat_history = [
    HumanMessage(content="What are the Generative AI Practice Requirements?"),
    AIMessage(content=response)
]
with_chat_response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "What techniques are discussed in the Foundation Model selection and customization requirement?"
})
print(with_chat_response["answer"])

The techniques discussed in the Foundation Model selection and customization requirement include:

1. Few-shot prompting over Zero-shot prompting
2. Chain-of-thought (COT) prompting
3. Retrieval Augmented Generation (RAG)
4. Fine-tuning
5. Model Customization using Amazon Bedrock


#### 3. Agent RAG

We define a retrieval tool and Tavily tool (search engine).

In [27]:
retriever_tool = create_retriever_tool(
    retriever=final_retriever,
    name="aws_generative_ai_competency_search",
    description="Search for information about AWS Generative AI Competency. For any questions about AWS Generative AI Competency, you must use this tool!",
)
search_tool = TavilySearchResults()
tools = [retriever_tool, search_tool]

In [28]:
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
print(agent_executor.invoke({"input": "Who won the 2001 Copa America?"})["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2001 Copa America winner'}`


[{'url': 'https://www.footballdatabase.eu/en/competition/overall/6857-copa_america/2001', 'content': "Copa America2001\n23%Home victories\n15%Draws\n38%Neutral ground victories\n23%Away victories\nCopa America 2001's fixtures and results\nFinalGroup BGroup AGroup CQuarter-finalsSemi-finals3rd/4th placeFinal\nLatest transfers\nThe best XI according to FBDB Index\nYour best XI\nIndividual rankingsLast updated on 15/03/2024 at 17:45\nScorers\nAssists\nGames\nclean sheets\nFair play\nScorers as subs\nScorers + Assists\nNewcomers\nGames list\nRankings\nForm\nHome\nAway\nSecond legs\nAttendances\nOffense\nDefense\nFirst half\nSecond half\nFair play\nLongest streaksLast updated on 15/03/2024 at 17:45\nWinning streak\nLoosing streak\nDrawing streak\nDrawless streak\nUnbeaten streak\nWinless streak\nInvincibility streak\nGoalless streak\nConceding streak\nScoring streak

In [30]:
print(agent_executor.invoke({"input": "What are the Generative AI Practice Requirements?"})["output"])



> Entering new AgentExecutor chain...

Invoking: `aws_generative_ai_competency_search` with `{'query': 'Generative AI Practice Requirements'}`


Aws Partner Network - OFFERS -> Aws Generative Ai Competency
Aws Generative Ai Competency - HAS_DOCUMENT -> Service Offering Validation Checklist
Service Offering Validation Checklist - PUBLISHED_ON -> March 2024
Aws Specialization Programs - HAS_DOCUMENT -> Aws Competency Partner Validation Checklist
Aws Partners - APPLY_FOR -> Aws Generative Ai Competency
Aws Partners - CONTACT -> Pdr
Aws Partners - CONTACT -> Pdm
Aws Partners - COMPLETE -> Self-Assessment Spreadsheet
Aws Partners - SUBMIT_APPLICATION -> Apn Partner Central
Aws Partners - PREPARE -> Technical Validation
Aws Partners - USE -> Checklist
Aws Partners - HAVE -> Aws Certified Engineers/Architects
Aws Partners - HAVE -> Operations Manager
Aws Partners - HAVE -> Business Development Executive
Self-Assessment Spreadsheet - RELATED_TO -> Aws Specialization Designations
Self-Assessm

In [31]:
print(agent_executor.invoke({
    "chat_history": chat_history,
    "input": "What techniques are discussed in the Foundation Model selection and customization requirement?"
})["output"])



> Entering new AgentExecutor chain...

Invoking: `aws_generative_ai_competency_search` with `{'query': 'Foundation Model Selection and Customization'}`





text: 's business operations.



GENAIPR-002 - Foundation Model selection and customization

The partner can select and customize an appropriate Foundation Model using the AWS technologies (for example, Amazon Bedrock, SageMaker Jumpstart, or Containers) and considering at least four or more selection factors (including licensing, customer skills, output quality, context windows, latency, budget, customizations supported, etc.)
The partner must clearly describe the following requirements for a customized model:

The model type and the level of customizations and optimizations performed to the model. The partner should also facilitate the adaptation of the selected model for various downstream tasks, ensuring optimal performance and strategic alignment with the business needs, including the use of Foundation Models leveraging the 